In [1]:
# Automatically reload modules
%load_ext autoreload
%autoreload 2

from dbpd.gait_analysis import *
from dbpd.gait_analysis_config import *
from dbpd.imu_preprocessing import *

In [2]:
path_to_data =  '../../../tests/data'

In [3]:
input_path = os.path.join(path_to_data, '1.sensor_data', 'imu')
output_path = os.path.join(path_to_data, '2.preprocessed_data', 'gait')
config = PreprocessingConfig()
preprocess_imu_data(input_path, output_path, config)

In [4]:
input_path = os.path.join(path_to_data, '2.preprocessed_data', 'gait')
output_path = os.path.join(path_to_data, '3.extracted_features', 'gait')
config = GaitFeatureExtractionConfig()
#config.set_sampling_frequency(50)
extract_gait_features(input_path, output_path, config)

(725, 9)


In [5]:
input_path = os.path.join(path_to_data, '3.extracted_features', 'gait')
output_path = os.path.join(path_to_data, '4.predictions', 'gait')
path_to_classifier_input = os.path.join(path_to_data, '0.classifiers', 'gait')
config = GaitDetectionConfig()
detect_gait(input_path, output_path, path_to_classifier_input, config)

In [6]:
input_path = os.path.join(path_to_data, '2.preprocessed_data', 'gait')
output_path = os.path.join(path_to_data, '3.extracted_features', 'gait')
config = ArmSwingFeatureExtractionConfig()
extract_arm_swing_features(input_path, output_path, config)

In [7]:
input_path = os.path.join(path_to_data, '3.extracted_features', 'gait')
output_path = os.path.join(path_to_data, '4.predictions', 'gait')
path_to_classifier_input = os.path.join(path_to_data, '0.classifiers', 'gait')
config = ArmSwingDetectionConfig()
detect_arm_swing(input_path, output_path, path_to_classifier_input, config)

In [8]:
path_to_feature_input = os.path.join(path_to_data, '3.extracted_features', 'gait')
path_to_prediction_input = os.path.join(path_to_data, '4.predictions', 'gait')
output_path = os.path.join(path_to_data, '5.quantification', 'gait')
config = ArmSwingQuantificationConfig()
quantify_arm_swing(path_to_feature_input, path_to_prediction_input, output_path, config)